In [1]:
import pandas as pd
import os
import json
import torch
data_dir = "D:/study/nlpcc/traning_datasets"
if not os.path.exists(data_dir):
    pass

D:\tools\Anaconda\envs\pytorchdl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




生成nlpcct5任务中的标签统计结果
----------------------------------------


In [2]:
with open(os.path.join(data_dir, "train_set.json"),'r') as f:
    trlst = []
    for line in f.readlines():
        trlst.append(json.loads(line.strip()))

In [3]:
trlst = trlst[0]

In [23]:
label_file_path = r'./src/transformers/data/processors/label_dir/'
label1_path = r'./src\transformers\data\processors\label_dir/level1_label.txt'
label2_path = r'./src/transformers/data/processors/label_dir/level2_label.txt'
label3_path = r'./src/transformers/data/processors/label_dir/level3_label.txt'

In [53]:
with open(label1_path,'r',encoding='utf-8') as f:
    label1_lst = eval(f.read())
    
with open(label2_path,'r',encoding='utf-8') as f:
    label2_lst = eval(f.read())
with open(label3_path,'r',encoding='utf-8') as f:
    label3_lst = eval(f.read())

In [59]:

def mk_label_dict(label_lst,level):
    level = f'level{level}'
    label_dict = {}
    for label in label_lst:
        label_dict[label]=0
    for article in trlst:
        ar_lv_lst = article[level]
        for k in ar_lv_lst:
            label_dict[k]+=1
    return label_dict
label1_dict = mk_label_dict(label1_lst,1)
label2_dict = mk_label_dict(label2_lst,2)
label3_dict = mk_label_dict(label3_lst,3)

In [61]:
with open(label_file_path+f'level{1}_dict.json','w',encoding='utf-8') as f:
    f.write(json.dumps(label1_dict))
with open(label_file_path+f'level{2}_dict.json','w',encoding='utf-8') as f:
    f.write(json.dumps(label2_dict))
with open(label_file_path+f'level{3}_dict.json','w',encoding='utf-8') as f:
    f.write(json.dumps(label3_dict))


In [62]:
def get_pos_weight(num_labels):
    if num_labels == 21:
        with open(r'D:\study\nlpcct5\src\transformers\data\processors\label_dir\level1_dict.json', 'r') as f:
            level_dict = eval(f.read())

    if level_dict:
        pos_num_dict = {}
        for k, v in level_dict.items():
            pos_num_dict[k] = (90000 - v)/v
        return pos_num_dict
    return None

In [63]:
torch.tensor(list(get_pos_weight(21).values()))


tensor([  5.8718,   7.7125,  19.5011,   9.3974,  36.4220,  31.2928,   6.5150,
        138.5349,  15.5746,  31.6087,   1.2943,  47.8865,   1.5680,   0.9649,
          7.1551,   2.4275,   1.9786,  15.9492,   0.6533,  78.1557,  23.8207])



label处理
--------------------------------------

In [ ]:
xlnetbase_lr5_bs2_1024_lv1


In [126]:
import os
reuters_train_path = os.path.abspath(r"D:\study\data\reuters\training")
reuters_test_path = os.path.abspath(r"D:\study\data\reuters\test")
cat_path = os.path.abspath(r'D:\study\data\reuters\cats.txt')
train_idx = os.listdir(reuters_train_path)
test_idx = os.listdir(reuters_test_path)


In [168]:
len(test_idx+train_idx)
#10788

10788

In [164]:

def mk_cat_lst():
    with open(cat_path,'r',encoding='utf-8') as f:
        cats_lst = f.readlines()
    cats_lst = [cat.strip().split(' ') for cat in cats_lst]
    cats_dict = {}
    class_lst = []
    for cat in cats_lst:
        tp,idx = cat[0].split('/')
        level4 = cat[1:]
        cats_dict[idx]=[tp,level4]
        class_lst.extend(level4)
    return cats_dict,class_lst

In [165]:
def load_mess(set_type,idx_lst):
    cats_dict,class_lst = mk_cat_lst()
    doc = []
    for idx in idx_lst:
        try:
            with open(f'D:/study/data/reuters/{set_type}/{idx}','r',encoding='utf-8') as f:
                t_doc = f.readlines()
        except:
            with open(f'D:/study/data/reuters/{set_type}/{idx}','r') as f:
                t_doc = f.readlines()
        title = t_doc[0].strip().lower()
        abstract = t_doc[1:]
        abstract = [k.strip() for k in abstract]
        abstract = ''.join(abstract)
        doc.append({'title':title,'abstract':abstract,'level4':cats_dict[idx][1]})
    return doc,class_lst
train_doc,train_class = load_mess('training',train_idx)
test_doc,test_class = load_mess('test',test_idx)

In [166]:
class_lst = train_class + test_class
train_class = sorted(list(set(train_class)))
test_class = sorted(list(set(test_class)))
assert train_class==test_class,'error!train label!=test label'
with open('./src/transformers/data/processors/label_dir/reuters_label.txt','w',encoding='utf-8') as f:
    f.write(str(train_class))

In [167]:
import json
class_num_dict = {}
for cls in train_class:
    class_num_dict[cls]=0
for cls in class_lst:
    class_num_dict[cls]+=1
with open('./src/transformers/data/processors/label_dir/reuters_dict.json','w',encoding='utf-8') as f:
    f.write(json.dumps(class_num_dict))
    

In [170]:
with open('./train_set.json','w',encoding='utf-8') as f:
    f.write(json.dumps(train_doc))
with open('./test_set.json','w',encoding='utf-8') as f:
    f.write(json.dumps(test_doc))

In [171]:
test_doc

[{'title': 'asian exporters fear damage from u.s.-japan rift',
  'abstract': 'Mounting trade friction between theU.S. And Japan has raised fears among many of Asia\'s exportingnations that the row could inflict far-reaching economicdamage, businessmen and officials said.They told Reuter correspondents in Asian capitals a U.S.Move against Japan might boost protectionist sentiment in theU.S. And lead to curbs on American imports of their products.But some exporters said that while the conflict would hurtthem in the long-run, in the short-term Tokyo\'s loss might betheir gain.The U.S. Has said it will impose 300 mln dlrs of tariffs onimports of Japanese electronics goods on April 17, inretaliation for Japan\'s alleged failure to stick to a pact notto sell semiconductors on world markets at below cost.Unofficial Japanese estimates put the impact of the tariffsat 10 billion dlrs and spokesmen for major electronics firmssaid they would virtually halt exports of products hit by thenew taxes."

In [172]:
type(1)==int

True